In [1]:
import pandas as pd
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

Dataset: https://www.kaggle.com/c/fake-news/data#

In [2]:
df=pd.read_csv('train_fake_news.csv')

In [3]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df=df.dropna()

In [6]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [8]:
vocab_size=5000

In [24]:
X=df.title.reset_index(drop=True)
y=df['label']

In [13]:
import nltk
from nltk.corpus import stopwords
import gensim

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nguye\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [14]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [88]:
X.apply(gensim.utils.simple_preprocess)

0        [house, dem, aide, we, didn, even, see, comey,...
1        [flynn, hillary, clinton, big, woman, on, camp...
2                [why, the, truth, might, get, you, fired]
3        [civilians, killed, in, single, us, airstrike,...
4        [iranian, woman, jailed, for, fictional, unpub...
                               ...                        
18280    [rapper, trump, poster, child, for, white, sup...
18281    [playoffs, schedule, matchups, and, odds, the,...
18282    [macy, is, said, to, receive, takeover, approa...
18283    [nato, russia, to, hold, parallel, exercises, ...
18284                            [what, keeps, the, alive]
Name: title, Length: 18285, dtype: object

In [97]:
def porterstemmer(sentence):
    return [ps.stem(word) for word in sentence if not word in stopwords.words('english')]

In [99]:
X=X.apply(gensim.utils.simple_preprocess).apply(porterstemmer)

In [100]:
X

0        [hous, dem, aid, even, see, comey, letter, jas...
1        [flynn, hillari, clinton, big, woman, campu, b...
2                                [truth, might, get, fire]
3          [civilian, kill, singl, us, airstrik, identifi]
4        [iranian, woman, jail, fiction, unpublish, sto...
                               ...                        
18280     [rapper, trump, poster, child, white, supremaci]
18281    [playoff, schedul, matchup, odd, new, york, time]
18282    [maci, said, receiv, takeov, approach, hudson,...
18283      [nato, russia, hold, parallel, exercis, balkan]
18284                                         [keep, aliv]
Name: title, Length: 18285, dtype: object

In [56]:
def onehot(text):
    return one_hot(text,n=vocab_size)

In [106]:
def Join(sentence):
    return ' '.join(sentence)

In [108]:
X.apply(Join).apply(onehot)

0        [690, 1227, 3569, 1195, 2455, 3395, 917, 1342,...
1                 [4387, 246, 4751, 3934, 4463, 874, 2770]
2                                    [87, 513, 3146, 3353]
3                       [981, 431, 4440, 2726, 1960, 1543]
4        [130, 4463, 379, 1074, 2905, 1606, 4463, 1215,...
                               ...                        
18280                 [3020, 2158, 1107, 4832, 3521, 2791]
18281              [2264, 88, 3512, 4108, 3444, 174, 3523]
18282    [4540, 2843, 3454, 260, 3749, 602, 1174, 3444,...
18283                  [4090, 1624, 1411, 110, 3115, 3353]
18284                                          [1549, 309]
Name: title, Length: 18285, dtype: object

In [110]:
padded_X=pad_sequences(X.apply(Join).apply(onehot),maxlen=20)

In [111]:
padded_X

array([[   0,    0,    0, ..., 1342, 3224, 1213],
       [   0,    0,    0, ..., 4463,  874, 2770],
       [   0,    0,    0, ...,  513, 3146, 3353],
       ...,
       [   0,    0,    0, ..., 3444,  174, 3523],
       [   0,    0,    0, ...,  110, 3115, 3353],
       [   0,    0,    0, ...,    0, 1549,  309]])

In [114]:
from sklearn.model_selection import train_test_split

In [123]:
X_train,X_test,y_train,y_test=train_test_split(padded_X,y,test_size=0.2,random_state=0)

In [117]:
from tensorflow.keras.layers import Dropout

In [136]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=20))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [134]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout_2 (Dropout)         (None, 20, 40)            0         
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [137]:
model.fit(X_train,y_train,validation_split=0.1,epochs=10,batch_size=64)

Epoch 1/10
206/206 [==============================] - 3s 9ms/step - loss: 0.3470 - accuracy: 0.8320 - val_loss: 0.1995 - val_accuracy: 0.9084
Epoch 2/10
206/206 [==============================] - 1s 7ms/step - loss: 0.1580 - accuracy: 0.9400 - val_loss: 0.1876 - val_accuracy: 0.9221
Epoch 3/10
206/206 [==============================] - 1s 7ms/step - loss: 0.1194 - accuracy: 0.9556 - val_loss: 0.2585 - val_accuracy: 0.9098
Epoch 4/10
206/206 [==============================] - 1s 7ms/step - loss: 0.0952 - accuracy: 0.9667 - val_loss: 0.2328 - val_accuracy: 0.9125
Epoch 5/10
206/206 [==============================] - 1s 7ms/step - loss: 0.0770 - accuracy: 0.9737 - val_loss: 0.2282 - val_accuracy: 0.9146
Epoch 6/10
206/206 [==============================] - 1s 7ms/step - loss: 0.0596 - accuracy: 0.9801 - val_loss: 0.2676 - val_accuracy: 0.9111
Epoch 7/10
206/206 [==============================] - 2s 8ms/step - loss: 0.0477 - accuracy: 0.9839 - val_loss: 0.2832 - val_accuracy: 0.9057
Epoch 

In [156]:
y_pred=model.predict(X_test)

In [157]:
y_pred[y_pred>0.5]=1
y_pred[y_pred<=0.5]=0

In [158]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9237079573420837